# Registrations CSV to DB

In [1]:
from pathlib import Path
import pandas as pd

Reads the last csv file matching `glob('*.csv')`. This should select the one with the latest datestring (alpha sort).

In [2]:
data = [pd.read_csv(file) for file in Path('sourcedata').glob('*.csv')][-1]

## Demographics

In [3]:
demogs = pd.DataFrame(
    data.set_index([
        'Name (Last)', 'Name (First)'
    ])[[
        'Gender', 'Birth Date', 'Zip Code', 'Source', 'Contact Frequency',
        'Phone Number', 'Email Address', 'UCF Main Campus', 'Online-Only Studies',
    ]]
)

In [4]:
demogs.to_csv(Path('derivatives') / 'subject_demographics.csv')

## Contact History

#### Select and stack the data by indexing (LastName,FirstName) and taking the Contact Strings for each instance of a contact to each person

In [5]:
contact_history = pd.DataFrame(
    data.set_index(
        ['Name (Last)', 'Name (First)']
    )[
        [c for c in data.columns if c.startswith('Contact Entry')]
    ].stack()
)

#### Prune the ContactString(0) into its constituents, separated by `|` characters

In [6]:
new_cols = { 0:'Date', 1:'ContactBy', 2:'ForStudy', 3:'Status' }
for key in new_cols:
    contact_history[new_cols[key]] = contact_history[0].str.split('|').str[key]

In [7]:
contact_history = contact_history.drop(columns=0)

In [8]:
contact_history.to_csv(Path('derivatives') / 'contact_history.db')

## Convert to SQLAlchemy